In [5]:
import mlflow
import pandas as pd
import os
import sys
from mlflow.tracking import MlflowClient

# Add the src directory to the path. TEMPORARY FIX
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from src.models.utils.mlflow_manager import MLflowManager

experiment_name = "test_experiments"

mlflow_manager = MLflowManager(experiment_name=experiment_name)

client = MlflowClient()

# run = mlflow_manager.search_runs(134)
# # print(run)

# metrics_df = run.filter(regex="^metrics\.")
# print(metrics_df)

print(client.get_run("7068e92cd88145ccb23ced8cb2051640").data.metrics)

# metric_names = [m.key for m in client.get_run("7068e92cd88145ccb23ced8cb2051640").data.metrics]

# print("Metric names:", metric_names)



# # Obté totes les runs de l'experiment
# runs_df = mlflow.search_runs(experiment_ids=[experiment.experiment_id])

# # Mostra només les mètriques (i altres columnes si vols)
# metrics_df = runs_df.filter(regex="^metrics\.")

# # Guarda-ho a CSV o Excel
# metrics_df.to_csv("exported_metrics.csv", index=False)
# # També pots fer: metrics_df.to_excel("exported_metrics.xlsx", index=False)


{'validation_accuracy': 0.73739153, 'validation_f1': 0.74056822, 'validation_precision': 0.76037127, 'validation_recall': 0.73739153, 'validation_loss': 0.7755514294339158, 'train_accuracy': 0.80393887, 'train_f1': 0.82935894, 'train_precision': 0.89459598, 'train_recall': 0.80393887, 'train_loss': 0.7511205277477354}


In [6]:
from mlflow.tracking import MlflowClient
import pandas as pd

client = MlflowClient()
run_id = "7068e92cd88145ccb23ced8cb2051640"

# Si no saps quines mètriques hi ha:
# Això només et dona el darrer valor, però podem agafar-ne el nom
latest_metrics = client.get_run(run_id).data.metrics
metric_names = list(latest_metrics.keys())

# Recollim l'historial complet de cada mètrica
metrics_data = {}

for name in metric_names:
    history = client.get_metric_history(run_id, name)
    # Ordenar per epoch (pas) i extreure només els valors
    metrics_data[name] = [m.value for m in sorted(history, key=lambda x: x.step)]

# Suposem que totes les mètriques tenen el mateix nombre d'èpoques
num_epochs = len(next(iter(metrics_data.values())))
metrics_data["epoch"] = list(range(num_epochs))

# Convertim a DataFrame
df = pd.DataFrame(metrics_data)
df = df[["epoch"] + metric_names]  # Reordenar columnes

print(df.head())


   epoch  validation_accuracy  validation_f1  validation_precision  \
0      0             0.670094       0.668660              0.672467   
1      1             0.676689       0.672677              0.670739   
2      2             0.687223       0.684305              0.682287   
3      3             0.688914       0.685376              0.682970   
4      4             0.693526       0.690233              0.687898   

   validation_recall  validation_loss  train_accuracy  train_f1  \
0           0.670094         0.846808        0.686298  0.685003   
1           0.676689         0.840380        0.690130  0.688783   
2           0.687223         0.824270        0.693839  0.692538   
3           0.688914         0.823155        0.696662  0.695382   
4           0.693526         0.816688        0.698641  0.697396   

   train_precision  train_recall  train_loss  
0         0.684430      0.686298    0.819800  
1         0.688246      0.690130    0.815072  
2         0.691980      0.693839   

In [7]:
import mlflow
from mlflow.tracking import MlflowClient
import json

# Defineix la run
run_id = "7068e92cd88145ccb23ced8cb2051640"
client = MlflowClient()

# Noms de mètriques (els últims valors)
metric_names = list(client.get_run(run_id).data.metrics.keys())

metrics_history = {}
loss_history = None

for name in metric_names:
    history = client.get_metric_history(run_id, name)

    # Excloure objectes no escalaris (com la matriu de confusió)
    # Assumim que qualsevol valor que no sigui float o int s'ignora
    values = []
    for m in sorted(history, key=lambda x: x.step):
        if isinstance(m.value, (float, int)):
            values.append(m.value)
    
    # Identifiquem la pèrdua (loss)
    if "loss" in name.lower():
        loss_history = values
    else:
        metrics_history[name] = values

# JSON final
output = {
    "metrics_history": metrics_history,
    "loss_history": loss_history
}

# Guardar
with open("run_metrics.json", "w") as f:
    json.dump(output, f, indent=4)

print("JSON creat correctament com 'run_metrics.json'")


JSON creat correctament com 'run_metrics.json'
